GitHub  
https://github.com/deepfakes/faceswap  
論文  
https://arxiv.org/abs/2001.00179v3  

<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/faceswap_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 全てのセルを実行する前に  
### タイプ変更
ランタイム→ランタイムのタイプを変更→GPU  
### GoogleDriveをマウント
ドライブをマウントを選択→Googleドライブに接続  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GPU確認
!nvidia-smi

In [ ]:
# GitHubからコードをClone
%cd /content/
!git clone --depth 1 https://github.com/deepfakes/faceswap.git

In [ ]:
# ライブラリをインストール
%cd /content/faceswap/
!pip3 install -r _requirements_base.txt

!pip3 uninstall tensorflow -y
!pip3 install tensorflow==2.2.*

# トレーニング時はnumpy 1.19.5、それ以外は、1.21.5が必要
!pip3 uninstall numpy -y
!pip3 install numpy --upgrade

In [ ]:
!pip3 show tensorflow
!pip3 show numpy

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

使用する動画  
src  
https://pixabay.com/ja/videos/%E5%AD%90%E4%BE%9B-%E5%AD%90%E3%81%A9%E3%82%82%E3%81%AE%E7%AC%91%E9%A1%94-%E8%B5%A4%E3%81%A1%E3%82%83%E3%82%93-33631/
  
target  
https://pixabay.com/ja/videos/%E5%A5%B3%E6%80%A7-%E8%8B%A5%E3%81%84-%E7%BE%8E%E3%81%97%E3%81%84-%E8%A1%A8%E7%8F%BE-32387/  

# データ準備
始めに利用したい顔の動画をアップロードしてください。  
次に、顔を交換する動画をアップロードしてください。

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/
!mkdir faceswap
%cd /content/drive/MyDrive/Colab\ Notebooks/faceswap
!mkdir src src_frames src_extract target target_frames target_extract models
%cd /content/drive/MyDrive/Colab\ Notebooks/faceswap/src

import glob

if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/src/*.mp4')):
  # 顔を利用する動画をアップロード
  from google.colab import files
  src_uploaded = files.upload()
  src_uploaded = list(src_uploaded.keys())
  print(src_uploaded)
  src_file = src_uploaded[0]

In [ ]:
%cd /content/faceswap/

if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/src_frames/*.png')):
  # frameに分割
  src_video_path = '/content/drive/MyDrive/Colab\ Notebooks/faceswap/src/'+ src_file
  !ffmpeg -i $src_video_path /content/drive/MyDrive/Colab\ Notebooks/faceswap/src_frames/src-video-frame-%d.png

In [ ]:
%cd /content/faceswap/
# 顔抽出(extract)実行
if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/src_extract/*.png')):
  !python faceswap.py extract \
    -i /content/drive/MyDrive/Colab\ Notebooks/faceswap/src_frames \
    -o /content/drive/MyDrive/Colab\ Notebooks/faceswap/src_extract

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/faceswap/target/

if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/target/*.mp4')):
  # 顔を以外を利用する動画をアップロード
  from google.colab import files
  target_uploaded = files.upload()
  target_uploaded = list(target_uploaded.keys())
  print(target_uploaded)
  target_file = target_uploaded[0]

In [ ]:
%cd /content/faceswap/

if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/target_frames/*.png')):
  # frameに分割
  target_video_path = '/content/drive/MyDrive/Colab\ Notebooks/faceswap/target/'+ target_file
  !ffmpeg -i $target_video_path /content/drive/MyDrive/Colab\ Notebooks/faceswap/target_frames/target-video-frame-%d.png

In [ ]:
%cd /content/faceswap/
# 顔抽出(extract)実行
if 0 == len(glob.glob('/content/drive/MyDrive/Colab Notebooks/faceswap/target_extract/*.png')):
  !python faceswap.py extract \
    -i /content/drive/MyDrive/Colab\ Notebooks/faceswap/target_frames \
    -o /content/drive/MyDrive/Colab\ Notebooks/faceswap/target_extract

In [ ]:
# numpyのバージョン変更
!pip3 uninstall numpy -y
!pip3 install numpy==1.19.*

In [ ]:
%cd /content/faceswap/

# 学習実行
!python faceswap.py train \
  -A /content/drive/MyDrive/Colab\ Notebooks/faceswap/target_extract \
  -B /content/drive/MyDrive/Colab\ Notebooks/faceswap/src_extract \
  -m /content/drive/MyDrive/Colab\ Notebooks/faceswap/models

In [ ]:
!pip3 uninstall numpy -y
!pip3 install numpy --upgrade

In [ ]:
# 顔の変換
%cd /content/faceswap/

!python faceswap.py convert \
  -i /content/drive/MyDrive/Colab\ Notebooks/faceswap/target_frames \
  -o /content/drive/MyDrive/Colab\ Notebooks/faceswap/results \
  -m /content/drive/MyDrive/Colab\ Notebooks/faceswap/models

In [ ]:
# フレーム画像を動画に変換
%cd /content/faceswap/

!ffmpeg -i /content/drive/MyDrive/Colab\ Notebooks/faceswap/results/target-video-frame-%0d.png -c:v libx264 -vf "fps=25,format=yuv420p" /content/drive/MyDrive/Colab\ Notebooks/faceswap/out.mp4